# Bayesian Personalized Ranking
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
const source_name = "MLE.Ensemble";

In [ ]:
username = ""
task = ""

In [ ]:
import NBInclude: @nbinclude
import SparseArrays: sparse
@nbinclude("../Alpha.ipynb");

In [ ]:
function get_query_features(alphas::Vector{String})
    A = Matrix{Float32}(undef, num_items(), length(alphas))
    @tprogress Threads.@threads for i = 1:length(alphas)
        A[:, i] = read_recommendee_alpha(alphas[i], "all").rating
    end
    collect(A')
end

function get_implicit_features()
    df = get_recommendee_split("implicit")
    sparse(df.item, df.user, df.rating, num_items(), 1)
end

function get_explicit_features()
    df = get_recommendee_split("explicit")
    sparse(df.item, df.user, df.rating, num_items(), 1)
end

function get_user_features()
    collect(vcat(get_implicit_features(), get_explicit_features()))
end

function get_embedding(
    u::Integer,
    a::Integer,
    q::Integer,
    user_features::AbstractMatrix,
    query_features::AbstractMatrix,
)
    user_features[:, u], [a], query_features[:, q]
end;

In [ ]:
function compute_alpha()
    params = read_params("$task/$source_name")
    U = get_user_features()
    Q = get_query_features(params["hyp"].alphas)
    Q = (Q .- params["inference_data"]["μ"]) ./ params["inference_data"]["σ"]
    m = params["m"]
    scores = Array{Float32}(undef, num_items())
    @tprogress Threads.@threads for i = 1:num_items()
        sample = get_embedding(1, i, i, U, Q)
        scores[i] = m(sample)[1]
    end
    write_recommendee_alpha(scores, "$task/$source_name")
end;

In [ ]:
compute_alpha();